<a href="https://colab.research.google.com/github/Mayank-004/VC-Industry-Predictive-analysis/blob/main/VC_Target_MOIC_KMEANS_CLUSTERING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the libraries

In [474]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
from sklearn.preprocessing import OrdinalEncoder


## Importing the dataset







In [475]:
url1 = 'https://raw.githubusercontent.com/Mayank-004/Project-in-python/main/VC_Exits.csv'
dataset1 = pd.read_csv(url1, error_bad_lines=False)

#url2 = 'https://raw.githubusercontent.com/Mayank-004/Project-in-python/main/VCExit2.csv'
#dataset2 = pd.read_csv(url2, error_bad_lines=False)

url3 = 'https://raw.githubusercontent.com/Mayank-004/Project-in-python/main/VCData2.csv'
dataset3 = pd.read_csv(url3, error_bad_lines=False)

dataset = pd.merge(dataset1,dataset3,left_on=["Company PBID"],right_on=["Company ID"])



dataset


,Company PBID,Exit Date,Exit Size (millions),Pre Value (millions),Post Value (millions),Exit Type,Last VC Deal Type,Total VC Capital Raised ($M)***,Total Preferred Capital Raised ($M)*,% Preferred Capital Raised*,Total Preferred Capital Raised ($M) - Combination**,% Preferred Capital Raised - Combination**,MOIC,Industry Sector,Industry Group,Industry Code,Verticals_x,City,State,Country,Deal ID,Company ID,Primary Industry Sector,Primary Industry Group,Primary Industry Code,All Industries,Verticals_y,Keywords,Current Financing Status,Current Business Status,Universe,CEO PBId,CEO Education,Deal No.,Deal ID.1,Announced Date,Deal Date,Deal Size,Deal Size Status,Pre-money Valuation,...,# Investors,# New Investors,# Follow-on Investors,Implied EV,Revenue,Revenue Growth since last debt deal,Gross Profit,Net Income,EBITDA,EBIT,Total Debt (from financials),Fiscal Year,Valuation/EBITDA,Valuation/EBIT,Valuation/Net Income,Valuation/Revenue,Valuation/Cash Flow,Deal Size/EBITDA,Deal Size/EBIT,Deal Size/Net Income,Deal Size/Revenue,Deal Size/Cash Flow,Debt/EBITDA,Debt/Equity,Implied EV/EBITDA,Implied EV/EBIT,Implied EV/Net Income,Implied EV/Revenue,Implied EV/Cash Flow,EBITDA Margin %,Current Employees,Native Currency of Deal,HQ Location,HQ Global Region,HQ Global Sub Region,Company City,Company State/Province,Company Post Code,Company Country,Year Founded
0,155748-70,11/15/17,NaN,NaN,NaN,Merger/Acquisition,Early Stage VC,NaN,NaN,NaN,NaN,NaN,NaN,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,Utah,United States,67911-49T,155748-70,Information Technology,Software,Other Software,"Business/Productivity Software, Other Software*","3D Printing, CloudTech & DevOps, TMT","additives manufacturing software, additives pr...",Formerly VC-backed,Generating Revenue,Venture Capital,131029-48P,"Texas A&M University, Ph.D. (Doctor of Philoso...",1,67911-49T,NaN,01-Jan-2015,0.33,Actual,NaN,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,US Dollars (USD),"Park City, UT",Americas,North America,Park City,Utah,84098,United States,2014.0
1,155748-70,11/15/17,NaN,NaN,NaN,Merger/Acquisition,Early Stage VC,NaN,NaN,NaN,NaN,NaN,NaN,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,Utah,United States,122433-04T,155748-70,Information Technology,Software,Other Software,"Business/Productivity Software, Other Software*","3D Printing, CloudTech & DevOps, TMT","additives manufacturing software, additives pr...",Formerly VC-backed,Generating Revenue,Venture Capital,131029-48P,"Texas A&M University, Ph.D. (Doctor of Philoso...",2,122433-04T,NaN,26-May-2015,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,US Dollars (USD),"Park City, UT",Americas,North America,Park City,Utah,84098,United States,2014.0
2,160695-55,1/13/17,NaN,NaN,NaN,Merger/Acquisition,Early Stage VC,$0.70,NaN,NaN,NaN,NaN,NaN,Information Technology,Software,Software Development Applications,"SaaS, TMT, CloudTech & DevOps",Tel Aviv,NaN,Israel,71323-57T,160695-55,Information Technology,Software,Software Development Applications,"Application Software, Software Development App...","CloudTech & DevOps, SaaS, TMT","a/b testing, application development, applicat...",Formerly VC-backed,Generating Revenue,Venture Capital,56786-14P,"BA (Bachelor of Arts), 2005, Business & Marketing",1,71323-57T,NaN,18-Mar-2016,0.70,Actual,NaN,...,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,US Dollars (USD),"Tel Aviv, Israel",Middle East,Middle East,Tel Aviv,NaN,NaN,Israel,2015.0
3,81857-08,3/18/19,$166.00,NaN,$166.00,Merger/Acquisition,Early Stage VC,$10.10,NaN,NaN,NaN,NaN,16.44x,Information Technology,Software,Business/Productivity Software,"SaaS, CloudTech & DevOps",Georgetown,Texas,United States,46035

### Filling missing values of Target column MOIC

In [476]:
# target column MOIC is derived by Exit Size divided by Total VC capital raised
dataset['Total VC Capital Raised ($M)***']= dataset['Total VC Capital Raised ($M)***'].str.replace("$","") # removing $ from Total VC capital
dataset['Total VC Capital Raised ($M)***']= dataset['Total VC Capital Raised ($M)***'].str.replace(",","") # removing "," from Total VC capital
dataset.loc[:,'Total VC Capital Raised ($M)***'] = dataset.loc[:,'Total VC Capital Raised ($M)***'].astype(float) # converting Total VC capital to float from string
dataset['Total VC Capital Raised ($M)***']
dataset['Exit Size (millions)']= dataset['Exit Size (millions)'].str.replace("$","") # removing $ from Exit Size
dataset['Exit Size (millions)']= dataset['Exit Size (millions)'].str.replace(",","") # removing "," from Exit Size
dataset.loc[:,'Exit Size (millions)'] = dataset.loc[:,'Exit Size (millions)'].astype(float) # converting Total VC capital to float from string

dataset['MOIC']= dataset['MOIC'].str.replace("x","") # removing $ from MOIC
dataset['MOIC']= dataset['MOIC'].str.replace(",","") # removing ',' from MOIC
dataset.loc[:,'MOIC'] = dataset.loc[:,'MOIC'].astype(float) # converting MOIC to float from string

#dataset=dataset.drop(columns=['Exit Size (millions)','Total VC Capital Raised ($M)***'])
# Drop duplicate columns  

dataset=dataset.drop(columns=['Deal ID','Company ID','Primary Industry Sector','Primary Industry Group','Primary Industry Code','All Industries','Verticals_y','HQ Location','HQ Global Region','HQ Global Sub Region','Company City',		'Company Country'])

# Drop Column intuitively which may not have significant impact
dataset=dataset.drop(columns=['Company PBID','Exit Date','Post Value (millions)','Native Currency of Deal','Keywords','Deal No.','Deal ID.1','Deal Date','Deal Class','Deal Status','CEO PBId'])


# calculating total working duration of company


from datetime import date 
todays_date = date.today() 

ts =todays_date.year

dataset['Total duration for Company'] = ts - dataset['Year Founded']

dataset=dataset.drop(columns=['Year Founded'])

dataset['Total duration for Company'] 

# removing NA values from MOIC

dataset.dropna(subset = ["MOIC"], inplace=True) 


In [477]:
# check missing value in dataset
missing = dataset.isna().sum().sort_values(ascending = False)
percent_missing = ((missing / dataset.isnull().count()) * 100).sort_values(ascending = False)
missing_df = pd.concat([missing,percent_missing], axis = 1, keys = ['Total', 'Percent'],sort = False)
missing_df[missing_df['Total'] >= 1]

,Total,Percent
Contingent Payout,108,100.000000
Implied EV,108,100.000000
Deal Type 3,108,100.000000
Debt Amount 3,108,100.000000
Implied EV/Cash Flow,108,100.000000
...,...,...
Company Post Code,11,10.185185
Deal Size Status,9,8.333333
Deal Size,9,8.333333
Raised to Date,4,3.703704


Drop Column which has missing values more than 5%

---



In [478]:
missing_df = missing_df.reset_index()
over30 = list(missing_df.loc[missing_df['Percent'] >= 5,'index'])
dataset = dataset.drop(over30, axis = 1)

In [479]:
dataset

,Exit Size (millions),Exit Type,Last VC Deal Type,Total VC Capital Raised ($M)***,MOIC,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Raised to Date,Deal Type,Business Status,Financing Status,Total duration for Company
3,166.00,Merger/Acquisition,Early Stage VC,10.10,16.44,Information Technology,Software,Business/Productivity Software,"SaaS, CloudTech & DevOps",Georgetown,United States,Formerly VC-backed,Generating Revenue,Venture Capital,10.10,Early Stage VC,Generating Revenue,Venture Capital-Backed,8.0
17,144.10,Merger/Acquisition,Series B,37.80,3.81,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,3.30,Seed Round,Stealth,Venture Capital-Backed,6.0
18,144.10,Merger/Acquisition,Series B,37.80,3.81,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,17.80,Early Stage VC,Generating Revenue,Venture Capital-Backed,6.0
19,144.10,Merger/Acquisition,Series B,37.80,3.81,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,37.80,Early Stage VC,Generating Revenue,Venture Capital-Backed,6.0
21,3258.00,Merger/Acquisition,Series F,364.50,8.94,Information Technology,Software,Business/Productivity Software,"SaaS, TMT, CloudTech & DevOps",San Francisco,United States,Formerly VC-backed,Generating Revenue,"Debt Financed, Private Equity, Publicly Listed...",335.00,Later Stage VC,Generating Revenue,Venture Capital-Backed,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,156.37,IPO,Series F,259.38,0.60,Information Technology,Software,Business/Productivity Software,"SaaS, TMT, CloudTech & DevOps",Santa Clara,United States,Formerly VC-backed,Generating Revenue,"Publicly Listed, Venture Capital",260.00,Later Stage VC,Generating Revenue,Venture Capital-Backed,13.0
297,40.00,Merger/Acquisition,Series F,183.19,0.22,Information Technology,Software,Communication Software,"SaaS, TMT, CloudTech & DevOps",Hackensack,United States,Formerly VC-backed,Generating Revenue,"Debt Financed, Venture Capital",165.10,Later Stage VC,Generating Revenue,Venture Capital-Backed,16.0
298,40.00,Merger/Acquisition,Series F,183.19,0.22,Information Technology,Software,Communication Software,"SaaS, TMT, CloudTech & DevOps",Hackensack,United States,Formerly VC-backed,Generating Revenue,"Debt Financed, Venture Capital",139.00,Later Stage VC,Generating Revenue,Venture Capital-Backed,16.0
303,8873.17,IPO,Series D,160.75,55.20,Information Technology,Software,Communication Software,"SaaS, Mobile, TMT, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Profitable,"Publicly Listed, Venture Capital",160.75,Later Stage VC,Generating Revenue,Venture Capital-Backed,10.0


## Filling Missing Values

In [485]:
# check missing value in df
missing = dataset.isna().sum().sort_values(ascending = False)
percent_missing = ((missing / dataset.isnull().count()) * 100).sort_values(ascending = False)
missing_df = pd.concat([missing,percent_missing], axis = 1, keys = ['Total', 'Percent'],sort = False)
missing_df[missing_df['Total'] >= 1]

,Total,Percent


In [483]:
# filling missing values in all other columns with mean 

dataset = dataset.fillna((dataset.mean()))

dataset.dropna(subset = ["Financing Status"], inplace=True)
#dataset.apply(lambda x: x.fillna(x.mean()),axis=0)

dataset['Raised to Date']= dataset['Raised to Date'].str.replace(",","")

dataset.loc[:,'Raised to Date'] = dataset.loc[:,'Raised to Date'].astype(float)

dataset['Raised to Date'] = dataset['Raised to Date'].fillna((dataset['Raised to Date'].mean()))

In [484]:
dataset

,Exit Size (millions),Exit Type,Last VC Deal Type,Total VC Capital Raised ($M)***,MOIC,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Raised to Date,Deal Type,Business Status,Financing Status,Total duration for Company
3,166.00,Merger/Acquisition,Early Stage VC,10.10,16.44,Information Technology,Software,Business/Productivity Software,"SaaS, CloudTech & DevOps",Georgetown,United States,Formerly VC-backed,Generating Revenue,Venture Capital,10.10,Early Stage VC,Generating Revenue,Venture Capital-Backed,8.0
17,144.10,Merger/Acquisition,Series B,37.80,3.81,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,3.30,Seed Round,Stealth,Venture Capital-Backed,6.0
18,144.10,Merger/Acquisition,Series B,37.80,3.81,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,17.80,Early Stage VC,Generating Revenue,Venture Capital-Backed,6.0
19,144.10,Merger/Acquisition,Series B,37.80,3.81,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,37.80,Early Stage VC,Generating Revenue,Venture Capital-Backed,6.0
21,3258.00,Merger/Acquisition,Series F,364.50,8.94,Information Technology,Software,Business/Productivity Software,"SaaS, TMT, CloudTech & DevOps",San Francisco,United States,Formerly VC-backed,Generating Revenue,"Debt Financed, Private Equity, Publicly Listed...",335.00,Later Stage VC,Generating Revenue,Venture Capital-Backed,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,156.37,IPO,Series F,259.38,0.60,Information Technology,Software,Business/Productivity Software,"SaaS, TMT, CloudTech & DevOps",Santa Clara,United States,Formerly VC-backed,Generating Revenue,"Publicly Listed, Venture Capital",260.00,Later Stage VC,Generating Revenue,Venture Capital-Backed,13.0
297,40.00,Merger/Acquisition,Series F,183.19,0.22,Information Technology,Software,Communication Software,"SaaS, TMT, CloudTech & DevOps",Hackensack,United States,Formerly VC-backed,Generating Revenue,"Debt Financed, Venture Capital",165.10,Later Stage VC,Generating Revenue,Venture Capital-Backed,16.0
298,40.00,Merger/Acquisition,Series F,183.19,0.22,Information Technology,Software,Communication Software,"SaaS, TMT, CloudTech & DevOps",Hackensack,United States,Formerly VC-backed,Generating Revenue,"Debt Financed, Venture Capital",139.00,Later Stage VC,Generating Revenue,Venture Capital-Backed,16.0
303,8873.17,IPO,Series D,160.75,55.20,Information Technology,Software,Communication Software,"SaaS, Mobile, TMT, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Profitable,"Publicly Listed, Venture Capital",160.75,Later Stage VC,Generating Revenue,Venture Capital-Backed,10.0


## Encoding categorical data

Label Encoding of Categorical Variable

In [486]:
#Encoder
def label_encoder(dataset):

    #Takes in dataset, returns label encoded dataframe and the label encoder models
    dataset_lencoded = dataset.copy(deep=True);
    label_encoders = [None]*dataset.shape[1];

    for i in range(dataset.shape[1]):
        le = LabelEncoder()
        dataset_lencoded.iloc[:,i] = le.fit_transform(dataset_lencoded.iloc[:,i])
        label_encoders[i] = le
    return dataset_lencoded, label_encoders


#Decoder
def label_decoder(dataset_lencoded, label_encoders):

    #Takes in encoded dataframe and list of encoders and returns decoded dataset
    #Label encoder is a list of label encoder models pertaining to each column
    for i in range(dataset_lencoded.shape[1]):
        dataset_lencoded.iloc[:,i] = label_encoders[i].inverse_transform(dataset_lencoded.iloc[:,i])

    return dataset_lencoded #Now it is decoded

In [487]:
dataset,x = label_encoder(dataset)
dataset

,Exit Size (millions),Exit Type,Last VC Deal Type,Total VC Capital Raised ($M)***,MOIC,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Raised to Date,Deal Type,Business Status,Financing Status,Total duration for Company
3,21,2,1,12,37,1,3,2,13,9,7,1,0,7,29,2,0,4,3
17,17,2,6,21,15,1,3,9,1,20,7,1,0,7,15,5,4,4,1
18,17,2,6,21,15,1,3,9,1,20,7,1,0,7,34,2,0,4,1
19,17,2,6,21,15,1,3,9,1,20,7,1,0,7,47,2,0,4,1
21,40,2,12,44,26,1,3,2,18,19,7,1,0,0,89,4,0,4,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,19,1,12,40,1,1,3,2,18,22,7,1,0,6,87,4,0,4,9
297,8,2,12,39,0,1,3,3,18,11,7,1,0,2,78,4,0,4,12
298,8,2,12,39,0,1,3,3,18,11,7,1,0,2,72,4,0,4,12
303,46,1,9,36,44,1,3,3,16,20,7,1,2,6,75,4,0,4,6


### Building Models

In [488]:
 import matplotlib.pyplot as plt
 #from kneed import kneeLocator
 from sklearn.cluster import KMeans
 from sklearn.metrics import silhouette_score
 from sklearn.preprocessing import StandardScaler

In [489]:
kmeans = KMeans(
            init="random",
            n_clusters=3,
            n_init=10,
            max_iter=300,
            random_state=42
  )

In [490]:
kmeans.fit(dataset)

KMeans(algorithm='auto', copy_x=True, init='random', max_iter=300, n_clusters=3,
       n_init=10, n_jobs=None, precompute_distances='auto', random_state=42,
       tol=0.0001, verbose=0)

In [491]:
dataset

,Exit Size (millions),Exit Type,Last VC Deal Type,Total VC Capital Raised ($M)***,MOIC,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Raised to Date,Deal Type,Business Status,Financing Status,Total duration for Company
3,21,2,1,12,37,1,3,2,13,9,7,1,0,7,29,2,0,4,3
17,17,2,6,21,15,1,3,9,1,20,7,1,0,7,15,5,4,4,1
18,17,2,6,21,15,1,3,9,1,20,7,1,0,7,34,2,0,4,1
19,17,2,6,21,15,1,3,9,1,20,7,1,0,7,47,2,0,4,1
21,40,2,12,44,26,1,3,2,18,19,7,1,0,0,89,4,0,4,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,19,1,12,40,1,1,3,2,18,22,7,1,0,6,87,4,0,4,9
297,8,2,12,39,0,1,3,3,18,11,7,1,0,2,78,4,0,4,12
298,8,2,12,39,0,1,3,3,18,11,7,1,0,2,72,4,0,4,12
303,46,1,9,36,44,1,3,3,16,20,7,1,2,6,75,4,0,4,6


In [492]:
kmeans.labels_


array([0, 0, 1, 1, 2, 0, 1, 2, 2, 2, 2, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       2, 2, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 2, 2, 0,
       1, 1, 1, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 0, 0, 1, 0, 0, 1, 1, 2, 0, 0,
       1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2],
      dtype=int32)

In [493]:
label_decoder(dataset,x)

,Exit Size (millions),Exit Type,Last VC Deal Type,Total VC Capital Raised ($M)***,MOIC,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Raised to Date,Deal Type,Business Status,Financing Status,Total duration for Company
3,166.00,Merger/Acquisition,Early Stage VC,10.10,16.44,Information Technology,Software,Business/Productivity Software,"SaaS, CloudTech & DevOps",Georgetown,United States,Formerly VC-backed,Generating Revenue,Venture Capital,10.10,Early Stage VC,Generating Revenue,Venture Capital-Backed,8.0
17,144.10,Merger/Acquisition,Series B,37.80,3.81,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,3.30,Seed Round,Stealth,Venture Capital-Backed,6.0
18,144.10,Merger/Acquisition,Series B,37.80,3.81,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,17.80,Early Stage VC,Generating Revenue,Venture Capital-Backed,6.0
19,144.10,Merger/Acquisition,Series B,37.80,3.81,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,37.80,Early Stage VC,Generating Revenue,Venture Capital-Backed,6.0
21,3258.00,Merger/Acquisition,Series F,364.50,8.94,Information Technology,Software,Business/Productivity Software,"SaaS, TMT, CloudTech & DevOps",San Francisco,United States,Formerly VC-backed,Generating Revenue,"Debt Financed, Private Equity, Publicly Listed...",335.00,Later Stage VC,Generating Revenue,Venture Capital-Backed,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,156.37,IPO,Series F,259.38,0.60,Information Technology,Software,Business/Productivity Software,"SaaS, TMT, CloudTech & DevOps",Santa Clara,United States,Formerly VC-backed,Generating Revenue,"Publicly Listed, Venture Capital",260.00,Later Stage VC,Generating Revenue,Venture Capital-Backed,13.0
297,40.00,Merger/Acquisition,Series F,183.19,0.22,Information Technology,Software,Communication Software,"SaaS, TMT, CloudTech & DevOps",Hackensack,United States,Formerly VC-backed,Generating Revenue,"Debt Financed, Venture Capital",165.10,Later Stage VC,Generating Revenue,Venture Capital-Backed,16.0
298,40.00,Merger/Acquisition,Series F,183.19,0.22,Information Technology,Software,Communication Software,"SaaS, TMT, CloudTech & DevOps",Hackensack,United States,Formerly VC-backed,Generating Revenue,"Debt Financed, Venture Capital",139.00,Later Stage VC,Generating Revenue,Venture Capital-Backed,16.0
303,8873.17,IPO,Series D,160.75,55.20,Information Technology,Software,Communication Software,"SaaS, Mobile, TMT, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Profitable,"Publicly Listed, Venture Capital",160.75,Later Stage VC,Generating Revenue,Venture Capital-Backed,10.0


In [494]:
dataset[kmeans.labels_==0]

,Exit Size (millions),Exit Type,Last VC Deal Type,Total VC Capital Raised ($M)***,MOIC,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Raised to Date,Deal Type,Business Status,Financing Status,Total duration for Company
3,166.00,Merger/Acquisition,Early Stage VC,10.10,16.44,Information Technology,Software,Business/Productivity Software,"SaaS, CloudTech & DevOps",Georgetown,United States,Formerly VC-backed,Generating Revenue,Venture Capital,10.10,Early Stage VC,Generating Revenue,Venture Capital-Backed,8.0
17,144.10,Merger/Acquisition,Series B,37.80,3.81,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,3.30,Seed Round,Stealth,Venture Capital-Backed,6.0
30,32.66,Merger/Acquisition,Series A,1.52,21.44,Information Technology,Software,Automation/Workflow Software,"TMT, CloudTech & DevOps",Grenoble,France,Formerly VC-backed,Generating Revenue,Venture Capital,1.52,Early Stage VC,Generating Revenue,Venture Capital-Backed,8.0
37,20.00,Merger/Acquisition,Seed Round,2.16,9.25,Information Technology,Software,Social/Platform Software,"Mobile, AdTech, TMT, CloudTech & DevOps",London,United Kingdom,Formerly VC-backed,Startup,"Private Equity, Venture Capital",2.16,Seed Round,Startup,Venture Capital-Backed,9.0
63,4.00,Merger/Acquisition,Early Stage VC,0.69,5.81,Information Technology,IT Services,IT Consulting and Outsourcing,"TMT, CloudTech & DevOps",Seattle,United States,Formerly VC-backed,Generating Revenue,"Debt Financed, Venture Capital",0.44,Seed Round,Generating Revenue,Venture Capital-Backed,15.0
64,4.00,Merger/Acquisition,Early Stage VC,0.69,5.81,Information Technology,IT Services,IT Consulting and Outsourcing,"TMT, CloudTech & DevOps",Seattle,United States,Formerly VC-backed,Generating Revenue,"Debt Financed, Venture Capital",0.69,Early Stage VC,Generating Revenue,Venture Capital-Backed,15.0
68,5.00,Merger/Acquisition,Later Stage VC,7.30,0.68,Information Technology,Software,Business/Productivity Software,"SaaS, Mobile, TMT, CloudTech & DevOps",Alpharetta,United States,Formerly VC-backed,Generating Revenue,Venture Capital,7.30,Later Stage VC,Startup,Venture Capital-Backed,13.0
74,70.00,Merger/Acquisition,Series A,2.80,25.00,Information Technology,Software,Network Management Software,"Cybersecurity, SaaS, TMT, CloudTech & DevOps",Sydney,Australia,Formerly VC-backed,Generating Revenue,Venture Capital,2.80,Early Stage VC,Generating Revenue,Venture Capital-Backed,6.0
94,60.00,Merger/Acquisition,Corporate,20.50,2.93,Information Technology,Software,Business/Productivity Software,"SaaS, TMT, CloudTech & DevOps",Rosh Ha'Ayin,Israel,Formerly VC-backed,Generating Revenue,"Debt Financed, Venture Capital",16.50,Early Stage VC,Generating Revenue,Venture Capital-Backed,10.0
98,42.00,Merger/Acquisition,Series A1,7.23,5.81,Information Technology,Software,Business/Productivity Software,"Cybersecurity, TMT, CloudTech & DevOps",Newton,United States,Formerly VC-backed,Generating Revenue,Venture Capital,7.23,Early Stage VC,Startup,Venture Capital-Backed,10.0


In [495]:
dataset[kmeans.labels_==1]

,Exit Size (millions),Exit Type,Last VC Deal Type,Total VC Capital Raised ($M)***,MOIC,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Raised to Date,Deal Type,Business Status,Financing Status,Total duration for Company
18,144.10,Merger/Acquisition,Series B,37.80,3.81,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,17.800000,Early Stage VC,Generating Revenue,Venture Capital-Backed,6.000000
19,144.10,Merger/Acquisition,Series B,37.80,3.81,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,37.800000,Early Stage VC,Generating Revenue,Venture Capital-Backed,6.000000
31,67.00,Merger/Acquisition,Series B,22.00,3.05,Information Technology,Software,Database Software,"SaaS, TMT, CloudTech & DevOps",Santa Clara,United States,Formerly VC-backed,Generating Revenue,Venture Capital,22.000000,Early Stage VC,Generating Revenue,Venture Capital-Backed,8.000000
38,239.69,Merger/Acquisition,Series C1,105.73,2.27,Information Technology,Software,Educational Software,"EdTech, Mobile, TMT, CloudTech & DevOps",Beijing,China,Formerly VC-backed,Generating Revenue,Venture Capital,9.590000,Early Stage VC,Generating Revenue,Venture Capital-Backed,9.000000
39,239.69,Merger/Acquisition,Series C1,105.73,2.27,Information Technology,Software,Educational Software,"EdTech, Mobile, TMT, CloudTech & DevOps",Beijing,China,Formerly VC-backed,Generating Revenue,Venture Capital,33.120000,Early Stage VC,Generating Revenue,Venture Capital-Backed,9.000000
40,239.69,Merger/Acquisition,Series C1,105.73,2.27,Information Technology,Software,Educational Software,"EdTech, Mobile, TMT, CloudTech & DevOps",Beijing,China,Formerly VC-backed,Generating Revenue,Venture Capital,62.320000,Later Stage VC,Generating Revenue,Venture Capital-Backed,9.000000
41,239.69,Merger/Acquisition,Series C1,105.73,2.27,Information Technology,Software,Educational Software,"EdTech, Mobile, TMT, CloudTech & DevOps",Beijing,China,Formerly VC-backed,Generating Revenue,Venture Capital,105.730000,Later Stage VC,Generating Revenue,Venture Capital-Backed,9.000000
72,24.00,Buyout/LBO,Later Stage VC,30.64,0.98,Information Technology,Computer Hardware,"Computers, Parts and Peripherals","SaaS, Internet of Things, TMT, CloudTech & DevOps",Fishers,United States,Private Equity-Backed,Generating Revenue,"Debt Financed, Private Equity, Venture Capital",26.900000,Later Stage VC,Generating Revenue,Venture Capital-Backed,11.000000
73,24.00,Buyout/LBO,Later Stage VC,30.64,0.98,Information Technology,Computer Hardware,"Computers, Parts and Peripherals","SaaS, Internet of Things, TMT, CloudTech & DevOps",Fishers,United States,Private Equity-Backed,Generating Revenue,"Debt Financed, Private Equity, Venture Capital",32.180000,Later Stage VC,Generating Revenue,Venture Capital-Backed,11.000000
87,402.70,Merger/Acquisition,Series C,99.34,4.05,Information Technology,Software,Network Management Software,"SaaS, TMT, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,99.330000,Later Stage VC,Generating Revenue,Venture Capital-Backed,8.000000


In [452]:
dataset[kmeans.labels_==2]

,Exit Type,Last VC Deal Type,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Deal Type,Business Status,Financing Status,Total duration for Company
0,Merger/Acquisition,Early Stage VC,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Grant,Generating Revenue,Corporation,7.000000
1,Merger/Acquisition,Early Stage VC,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,7.000000
4,Merger/Acquisition,Seed Round,Information Technology,Software,Application Software,"Mobile, Artificial Intelligence & Machine Lear...",Minsk,Belarus,Formerly VC-backed,Startup,Venture Capital,Seed Round,Startup,Venture Capital-Backed,5.000000
17,Merger/Acquisition,Series B,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Seed Round,Stealth,Venture Capital-Backed,6.000000
18,Merger/Acquisition,Series B,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,6.000000
19,Merger/Acquisition,Series B,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,6.000000
27,Merger/Acquisition,Later Stage VC,Information Technology,Software,Software Development Applications,"Mobile, TMT, CloudTech & DevOps",Seattle,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Later Stage VC,Generating Revenue,Venture Capital-Backed,9.976667
28,Merger/Acquisition,Later Stage VC,Information Technology,Software,Software Development Applications,"Mobile, TMT, CloudTech & DevOps",Seattle,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Seed Round,Generating Revenue,Venture Capital-Backed,9.976667
37,Merger/Acquisition,Seed Round,Information Technology,Software,Social/Platform Software,"Mobile, AdTech, TMT, CloudTech & DevOps",London,United Kingdom,Formerly VC-backed,Startup,"Private Equity, Venture Capital",Seed Round,Startup,Venture Capital-Backed,9.000000
47,Merger/Acquisition,Later Stage VC,Information Technology,Software,Communication Software,SaaS,San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Later Stage VC,Generating Revenue,Venture Capital-Backed,12.000000
